# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.  
[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

2.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

3.  
[客户问题] 我看房价还在涨，这个投资回报怎么样？
[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。

4.  
[客户问题] 有没有学校？
[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。

5.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。

6.  
[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

7.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

8.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。

9.  
[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

10.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [1]:
with open("real_estate_sales_data.txt") as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [2]:
from langchain.text_splitter import CharacterTextSplitter

In [3]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [4]:
docs = text_splitter.create_documents([real_estate_sales])

Created a chunk of size 113, which is longer than the specified 100
Created a chunk of size 104, which is longer than the specified 100
Created a chunk of size 104, which is longer than the specified 100
Created a chunk of size 106, which is longer than the specified 100
Created a chunk of size 107, which is longer than the specified 100
Created a chunk of size 107, which is longer than the specified 100


In [5]:
docs[0]

Document(page_content='[客户问题] 船期安排是怎样的？\n[销售回答] 我们每周有定期的船期安排，针对不同的航线和目的港，具体的船期可能会有所不同。我可以根据您的货物目的地给您提供最接近的几个船期选项。')

In [6]:
len(docs)

43

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
query = "贵公司的订舱流程是怎样的"

In [9]:
answer_list = db.similarity_search(query)

In [10]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 贵公司的订舱流程是怎样的？
[销售回答] 订舱流程非常简单。首先，您提供货物详情和预期的发货日期。然后，我们提供航班选项和报价。一旦确认，我们会为您安排空间并处理相关的运输文件。

[客户问题] 我能在网上直接订舱吗？
[销售回答] 是的，我们提供在线订舱服务。您可以通过我们的官方网站进行订舱，系统会引导您完成整个流程。当然，我们的客户服务团队也随时准备帮助您。

[客户问题] 你们如何处理紧急订舱？
[销售回答] 对于紧急的订舱需求，我们会优先处理并尽一切可能满足您的要求。请尽快提供您的货物细节和急需的时间框架，我们将全力以赴。

[客户问题] 我怎样才能确认我的订舱已被接受？
[销售回答] 一旦我们处理完您的订舱，您将收到一封确认邮件，其中包含订舱详情和参考号码。您随时可以使用这些信息查询您的订舱状态。



In [11]:
db.save_local("real_estates_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [12]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [13]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001F1FD9BD3C0>, search_kwargs={'k': 3})

In [14]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[客户问题] 贵公司的订舱流程是怎样的？
[销售回答] 订舱流程非常简单。首先，您提供货物详情和预期的发货日期。然后，我们提供航班选项和报价。一旦确认，我们会为您安排空间并处理相关的运输文件。

[客户问题] 我能在网上直接订舱吗？
[销售回答] 是的，我们提供在线订舱服务。您可以通过我们的官方网站进行订舱，系统会引导您完成整个流程。当然，我们的客户服务团队也随时准备帮助您。

[客户问题] 你们如何处理紧急订舱？
[销售回答] 对于紧急的订舱需求，我们会优先处理并尽一切可能满足您的要求。请尽快提供您的货物细节和急需的时间框架，我们将全力以赴。



In [15]:
docs = topK_retriever.get_relevant_documents("你们可以提供门到门的服务吗")

In [16]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 你们可以提供门到门的服务吗？
[销售回答] 是的，我们提供全方位的门到门物流解决方案，包括内陆运输、装卸、存储和海运等服务。您只需提供货物和收货地址，其余的交给我们。

[客户问题] 你们能处理关税和清关手续吗？
[销售回答] 是的，我们有提供一站式的清关服务，可以帮助您处理所有出口和进口的关税和清关手续。只需提供必要的文件，我们会处理其余事宜。

[客户问题] 你们能帮忙做货物的包装或标记吗？
[销售回答] 当然可以，我们提供货物的包装和标记服务。请告诉我们您的具体要求，我们将确保按照您的指示完成这些工作。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [17]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [18]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 贵公司的订舱流程是怎样的？
[销售回答] 订舱流程非常简单。首先，您提供货物详情和预期的发货日期。然后，我们提供航班选项和报价。一旦确认，我们会为您安排空间并处理相关的运输文件。

[客户问题] 我能在网上直接订舱吗？
[销售回答] 是的，我们提供在线订舱服务。您可以通过我们的官方网站进行订舱，系统会引导您完成整个流程。当然，我们的客户服务团队也随时准备帮助您。



### 提取向量数据库中的`销售回答`

In [19]:
docs = retriever.get_relevant_documents(query)

In [20]:
docs[0].page_content

'[客户问题] 贵公司的订舱流程是怎样的？\n[销售回答] 订舱流程非常简单。首先，您提供货物详情和预期的发货日期。然后，我们提供航班选项和报价。一旦确认，我们会为您安排空间并处理相关的运输文件。'

In [21]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 贵公司的订舱流程是怎样的？\n',
 '订舱流程非常简单。首先，您提供货物详情和预期的发货日期。然后，我们提供航班选项和报价。一旦确认，我们会为您安排空间并处理相关的运输文件。']

In [22]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [23]:
ans

'订舱流程非常简单。首先，您提供货物详情和预期的发货日期。然后，我们提供航班选项和报价。一旦确认，我们会为您安排空间并处理相关的运输文件。'

#### 尝试各种问题

In [24]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [25]:
query = "能否提供包装服务"

print(sales(query))

['当然可以。我们提供专业的包装服务，确保您的货物在运输过程中得到最好的保护。请告诉我您的具体包装要求。\n\n绝对可以，以下是更多的订舱话术：', '当然可以，我们提供货物的包装和标记服务。请告诉我们您的具体要求，我们将确保按照您的指示完成这些工作。']


In [26]:
print(sales(query, 0.75))

['当然可以。我们提供专业的包装服务，确保您的货物在运输过程中得到最好的保护。请告诉我您的具体包装要求。\n\n绝对可以，以下是更多的订舱话术：', '当然可以，我们提供货物的包装和标记服务。请告诉我们您的具体要求，我们将确保按照您的指示完成这些工作。', '我们的附加值服务包括但不限于货物保险、专业包装、仓储、分拨、配送和供应链管理。根据您的需求，我们可以提供定制的解决方案。', '是的，我们有提供冷藏集装箱的服务，能够确保您的货物在合适的温度下运输。请提供您的具体要求，我会助您安排。']


In [28]:
query = "如何反馈？"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []

score:0.75 ans: ['我们非常重视客户的反馈。如果您对我们的服务有任何不满，请直接联系客户服务部门或通过我们的官方投诉渠道提出，我们承诺会尽快解决您的问题。']

score:0.5 ans: ['我们非常重视客户的反馈。如果您对我们的服务有任何不满，请直接联系客户服务部门或通过我们的官方投诉渠道提出，我们承诺会尽快解决您的问题。', '如果您发现货物有短缺，请尽快联系我们，并提供相关的运输和交货记录。我们将调查此事并寻求相应的解决方案。', '我们会尽全力确保按时交货，但如果发生延误，我们的客户服务团队将及时与您沟通，并为您提供解决方案和必要的援助。', '一旦您的货物装船，我们将会发送货物装船通知给您，您也可以通过我们的在线系统随时查看货物状态。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [29]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [31]:
qa_chain({"query": "可以告诉我订舱的价格吗？"})

{'query': '可以告诉我订舱的价格吗？',
 'result': '对不起，我无法提供具体的订舱价格。您可以登录我们的在线客户服务门户查询实时的费率，那里会提供详细的价格信息。如果需要帮助，我们的客户服务团队也可以协助您了解费用详情。'}

In [32]:
qa_chain({"query": "在运输过程中，如果我需要额外的服务，应该怎么办？"})

{'query': '在运输过程中，如果我需要额外的服务，应该怎么办？',
 'result': '如果您在运输途中需要任何额外的服务，比如更改目的地或添加附加服务，只需联系我们的客服团队，我们会尽力协助您。'}

In [33]:
print(sales("在运输过程中，如果我需要额外的服务，应该怎么办？"))

['如果您在运输途中需要任何额外的服务，比如更改目的地或添加附加服务，只需联系我们的客服团队，我们会尽力协助您。', '绝对可以。我们提供货物保价服务，以确保在运输过程中由于意外造成的损失能得到赔偿。我将向您解释保价的流程和费用。']


## 加载 FAISS 向量数据库已有结果

In [34]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings(), allow_dangerous_deserialization=True)

In [35]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [37]:
qa_chain({"query": "你们怎么处理海关的问题"})

{'query': '你们怎么处理海关的问题',
 'result': '我们有专业的海关代理团队，他们对各国海关规定非常熟悉，并将根据您的货物类型和目的地为您处理所有必要的海关事宜。'}

In [38]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [39]:
qa_chain({"query": "你们怎么处理海关的问题"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '你们怎么处理海关的问题',
 'result': '我们有专业的海关代理团队，他们对各国海关规定非常熟悉，并将根据您的货物类型和目的地为您处理所有必要的海关事宜。'}

In [40]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [41]:
result = qa_chain({"query": "我能在网上直接订舱吗？"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


In [42]:
result

{'query': '我能在网上直接订舱吗？',
 'result': '是的，您可以通过官方网站进行在线订舱，系统会引导您完成整个流程。同时，客户服务团队也随时准备帮助您。',
 'source_documents': [Document(page_content='[客户问题] 我能在网上直接订舱吗？\n[销售回答] 是的，我们提供在线订舱服务。您可以通过我们的官方网站进行订舱，系统会引导您完成整个流程。当然，我们的客户服务团队也随时准备帮助您。'),
  Document(page_content='[客户问题] 我能在网上查询费率和空间情况吗？\n[销售回答] 是的，您可以登录我们的在线客户服务门户，那里提供实时的费率查询和舱位预订服务，非常方便。')]}